## What is a JIT?

In [ ]:
def add(a, b):
    return a + b

In [ ]:
class Banana:
    def __init__(self, num=1):
        self.num = num
        
    def __add__(self, other):
        if type(other) == Banana:
            return Banana(self.num + other.num)
        else:
            return FruitSalad([self, other])


class Apple:
    def __init__(self, num=1):
        self.num = num
        
    def __add__(self, other):
        if type(other) == Apple:
            return Apple(self.num + other.num)
        else:
            return FruitSalad([self, other])

        
class FruitSalad:
    def __init__(self, fruits):
        self.fruits = fruits

In [ ]:
add(Apple(), Banana())

## Whiteboard: Sauvola thresholding

## Timing various bits of the inner loop

In [ ]:
import itertools
import numpy as np
from scipy import ndimage as ndi
from skimage.transform import integral_image
from skimage import util
from skimage.util import dtype_limits
import numba


@numba.jit(nopython=True, nogil=True, fastmath=True)
def _correlate_sparse_loop(input, indices, offsets,
                           values, output):
    for i, j in enumerate(indices):
        for off, val in zip(offsets, values):
            output[i] += input[j + off] * val


def correlate_sparse(image, kernel):
    indices = np.nonzero(kernel)
    offsets = np.ravel_multi_index(indices, image.shape)
    values = kernel[indices].astype(image.dtype)
    result = np.zeros([a - b + 1
                       for a, b in zip(image.shape, kernel.shape)],
                      dtype=image.dtype)
    corner_multi_indices = np.meshgrid(*[np.arange(i)
                                         for i in result.shape],
                                       indexing='ij',
                                       sparse=True)
    corner_indices = np.ravel_multi_index(corner_multi_indices,
                                          image.shape).ravel()
    _correlate_sparse_loop(
        image.ravel(), corner_indices, offsets, values,
        result.ravel()
    )
    return result

In [ ]:
image = np.random.random((4000, 6000))
w = 301
kern = np.zeros((w + 1,) * image.ndim)
for indices in itertools.product(*([[0, -1]] * image.ndim)):
    kern[indices] = (-1) ** (image.ndim % 2 != np.sum(indices) % 2)

In [ ]:
indices = np.nonzero(kern)
offsets = np.ravel_multi_index(indices, image.shape)
values = kern[indices].astype(image.dtype)
result = np.zeros([a - b + 1
                   for a, b in zip(image.shape, kern.shape)],
                  dtype=image.dtype)
corner_multi_indices = np.meshgrid(*[np.arange(i)
                                     for i in result.shape],
                                   indexing='ij',
                                   sparse=True)
corner_indices = np.ravel_multi_index(corner_multi_indices,
                                      image.shape).ravel()

In [ ]:
# warmup jit
_correlate_sparse_loop(
    image.ravel(), corner_indices, offsets, values,
    result.ravel()
)

In [ ]:
%%timeit -n 5 -r 5 -o
_correlate_sparse_loop(
    image.ravel(), corner_indices, offsets, values,
    result.ravel()
)

In [6]:
%%timeit -n 1 -r 1 -o
_correlate_sparse_loop.py_func(
    image.ravel(), corner_indices, offsets, values,
    result.ravel()
)

1min 20s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


<TimeitResult : 1min 20s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

In [ ]:
len(corner_indices), len(offsets)

In [ ]:
max(offsets)

In [ ]:
# M elems per second (Numba):
time = 0.540
len(corner_indices) * len(offsets) / time / 1e6

In [ ]:
# M elems per second (Python):
time = 75
len(corner_indices) * len(offsets) / time / 1e6

In [ ]:
@numba.jit(nopython=True, nogil=True, fastmath=True)
def _correlate_sparse_offsets(input, indices, offsets, values, output):
    for off, val in zip(offsets, values):
        # this loop order optimises cache access, gives up to 10x speedup
        for i, j in enumerate(indices):
            output[i] += input[j + off] * val 

In [ ]:
# warmup jit
_correlate_sparse_offsets(
    image.ravel(), corner_indices, offsets, values,
    result.ravel()
)

In [ ]:
%%timeit -n 5 -r 5
_correlate_sparse_offsets(
    image.ravel(), corner_indices, offsets, values,
    result.ravel()
)

In [ ]:
# M elems per second:
time = 0.132
len(corner_indices) * len(offsets) / time / 1e6

In [ ]:
def print_lines(substring, string):
    lines = string.split('\n')
    for line in lines:
        if substring in line:
            print(line)

In [ ]:
asm = list(_correlate_sparse_offsets.inspect_asm().values())[0]
print('single instructions:', asm.count('sd'))
print('packed instructions:', asm.count('pd'))
print_lines('sd', asm)

In [ ]:
@numba.jit(nopython=True, nogil=True, fastmath=True)
def _correlate_no_indirection(input, indices, offsets, values, output):
    for off, val in zip(offsets, values):
        for i in range(len(indices)):
            output[i] += input[i + off] * val


In [ ]:
# warmup jit
_correlate_no_indirection(
    image.ravel(), corner_indices, offsets, values,
    result.ravel()
)

In [ ]:
%%timeit -n 5 -r 5
_correlate_no_indirection(
    image.ravel(), corner_indices, offsets, values,
    result.ravel()
)

In [ ]:
# M elems per second:
time = 0.094
len(corner_indices) * len(offsets) / time / 1e6

In [ ]:
asm = list(_correlate_no_indirection.inspect_asm().values())[0]
print('single instructions:', asm.count('sd'))
print('packed instructions:', asm.count('pd'))
print_lines('pd', asm)

In [ ]:
@numba.jit(nopython=True, nogil=True, fastmath=True)
def _correlate_no_indir_offset(input, indices, offsets, values, output):
    for off, val in zip(offsets, values):
        for i in range(len(indices)):
            output[i] += input[i] * val


In [ ]:
# warmup jit
_correlate_no_indir_offset(
    image.ravel(), corner_indices, offsets, values,
    result.ravel()
)

In [ ]:
%%timeit -n 5 -r 5
_correlate_no_indir_offset(
    image.ravel(), corner_indices, offsets, values,
    result.ravel()
)

In [ ]:
asm = list(_correlate_no_indir_offset.inspect_asm().values())[0]
print('single instructions:', asm.count('sd'))
print('packed instructions:', asm.count('pd'))
print_lines('pd', asm)

In [ ]:
image32 = image.astype(np.float32)
result32 = result.astype(np.float32)
values32 = values.astype(np.float32)

In [ ]:
# warmup jit
_correlate_sparse_offsets(
    image32.ravel(), corner_indices, offsets, values32,
    result32.ravel()
)

In [ ]:
%%timeit -n 5 -r 5
_correlate_sparse_offsets(
    image32.ravel(), corner_indices, offsets, values32,
    result32.ravel()
)

In [ ]:
%%timeit -n 5 -r 5
_correlate_no_indir_offset(
    image32.ravel(), corner_indices, offsets, values32,
    result32.ravel()
)

In [ ]:
# M elems per second:
time = 0.049
len(corner_indices) * len(offsets) / time / 1e6

In [ ]:
asm = list(_correlate_no_indir_offset.inspect_asm().values())[1]
print('single instructions:', asm.count('ss'))
print('packed instructions:', asm.count('ps'))
print_lines('ps', asm)